**Source:** [Better Everyday] blog, by [Rob Go], cofounder of NextView.

Fragments of the original article are displayed as block quotations.

[Better Everyday]: https://bettereveryday.vc/how-hard-is-it-to-generate-a-10x-return-on-an-investment-9c1656d6c3af
[Rob Go]: https://bettereveryday.vc/@robgo

> How Hard Is It to Generate a 10X Return on an Investment?
> ===============================

> *Disclaimer: This post gets fairly detailed on venture math. I generally think it’s helpful for founders who take VC money to understand the business of their investors because it impacts their incentives and how they see the world. Those who have no interest in these sorts of topics should move on or go back to watching the price of BTC :)*

> Every time I see blog posts referencing multiples on VC investments, I wonder if writers or readers appreciate how hard it is to generate these kinds of multiples and how multiples should really be calculated.

> Obviously, the way to calculate a return multiple is to divide the amount returned from an investment by the dollars invested. If I invested \$10M in a company and got back \$100M, that’s a 10X return. Seems pretty straightforward, right?

> **The problem arises when multiples are inferred from incomplete data.** It’s quite rare that anybody but the fund manager actually knows the dollars out and dollars returned by a specific investment. And if you have incomplete data, there are usually a number of things that go into calculating or estimating the return. Here are a few other things to think about.

> ## How to Actually Calculate a Investment Return Multiple

> ### Remember each round of financing further dilutes early investors’ money.

> Let’s say a seed investor put in \$1M in a company’s first financing around at a \$10M post-money valuation. 
> The company ultimately sold for \$200M. So, one might infer that the seed investor made \$200M/\$10M= 20X their money.

**Task.** Define a function `IRM` that computes the *investment return multiple* of an investor given

  - its investment `I`

  - the post-money valuation `PMV`
  
  - the exit value `EV` (the price at which the company was sold).

In [1]:
def IRM(I, PMV, EV):
    return EV / PMV

**Task.** Check that your computation is consistent with the numbers of the article:

In [2]:
from ipywidgets import interact, FloatText
from IPython.display import Markdown

wI = FloatText(description="investment (M)", value=1.0)
wPMV = FloatText(description="post-money valuation (M)", value=10.0)
wEV = FloatText(description="exit value (M)", value=200.0)

@interact(I=wI, PMV=wPMV, EV=wEV, continuous_update=False)
def display_IRM(I, PMV, EV):
    multiple = IRM(I, PMV, EV)
    display(Markdown("**Investment Return Multiple:** {0:.1f}X".format(multiple)))

interactive(children=(FloatText(value=1.0, description='investment (M)'), FloatText(value=10.0, description='post-money valuation (M)'), FloatText(value=200.0, description='exit value (M)'), Output()), _dom_classes=('widget-interact',))

**Question.**
If the investment is 1M and the post-money valuation is 10M, 
what is the minimal exit value that meets the target of a 10x multiple?

**Answer.** The exit value should be at least **100M**. 
It's pretty clear that the return is an increasing function of the exit value and: 

In [3]:
display_IRM(I=1.0, PMV=10.0, EV=90)
display_IRM(I=1.0, PMV=10.0, EV=100)
display_IRM(I=1.0, PMV=10.0, EV=120)

**Investment Return Multiple:** 9.0X

**Investment Return Multiple:** 10.0X

**Investment Return Multiple:** 12.0X

**Question.** Does the investment return multiple depend on the investment value?

**Answer.** Nope. There is the following anecdotical evidence:

In [4]:
display_IRM(I=1.0, PMV=10.0, EV=100)
display_IRM(I=2.0, PMV=10.0, EV=100)
display_IRM(I=5.0, PMV=10.0, EV=100)
display_IRM(I=10.0, PMV=10.0, EV=100)

**Investment Return Multiple:** 10.0X

**Investment Return Multiple:** 10.0X

**Investment Return Multiple:** 10.0X

**Investment Return Multiple:** 10.0X

and then, of course the fact that the expression `EV / PMV` used in `IRM` doesn't use the variable `I`.

**Task.** Implement a conversion between the investment return multiple 
and the more familiar *return on investment* (ROI) percentage.

In [5]:
def IRM_to_ROI(IRM):
    return (IRM - 1.0) * 100.0

In [6]:
from ipywidgets import interact, FloatText
from IPython.display import Markdown

wIRM = FloatText(description="investment return multiple", value=10.0)

@interact(IRM=wIRM, continuous_update=False)
def display_ROI(IRM):
    ROI = IRM_to_ROI(IRM)
    display(Markdown("**Return On Investment:** {0:.1f}%".format(ROI)))

interactive(children=(FloatText(value=10.0, description='investment return multiple'), Output()), _dom_classes=('widget-interact',))

**Question.** What is the ROI associated to a 3X multiple ?

**Answer.** 200%, since:

In [7]:
display_ROI(3.0)

**Return On Investment:** 200.0%

> Unfortunately, this is almost never the case.

> In almost any outcome like this, the company goes on to raise more money, usually at higher prices. As new investors come on board, all existing shareholders of the company will have their ownership stake in the business diluted. This also happens when a company expands their option pool.

> Here’s how that plays out for the seed investors.

> Let’s say this theoretical company raises just one more round of financing. It’s a \$10M round at a \$50M post-money valuation. And as part of the round, the option pool of the company is expanded by an additional 10%. 
After this round, the company has their \$200M exit.

> In this example, the flow of the math would be something like this: The seed investor owned 10% after the seed financing. In the next round, the seed investor’s ownership is diluted by 30% (20% due the new financing and 10% due to the expanded option pool).

> So the seed investor’s ownership was actually 7% at exit. 
Thus, their \$1M returns \$14M — a 14X return. 
Pretty great, but meaningfully different from 20X.

**Task.** Create a new function `IRM` that computes the investment return multiple given:

  - your first round investment `I`
  
  - the first round post-money valuation `PMV1`
  
  - the second round size `RS`
  
  - the second round option pool `OP` (a percentage)
  
  - the second round post-money valuation `PMV2`
   
  - the exit value `EV`
  

In [8]:
def IRM(I, PMV1, OP, RS, PMV2, EV):
    ownership = I / PMV1
    dilution = RS / PMV2 + OP/100.0
    ownership = ownership * (1 - dilution)
    return (ownership * EV) / I

**Task.** Check with the widget below that your computation match the ones of the article.

In [11]:
from ipywidgets import interactive_output, FloatText, FloatSlider, Label, VBox, HBox
from IPython.display import Markdown

wL1 = Label("First Round")
wI = FloatText(description="investment (M)", value=1.0)
wPMV1 = FloatText(description="post-money valuation (M)", value=10.0)

wL2 = Label("Second Round")
wRS = FloatText(description="round size (M)", value=10.0)
wOP = FloatSlider(description="option pool (%)", min=0, max=100.0, value=10.0)
wPMV2 = FloatText(description="post-money valuation (M)", value=50.0)

def update_option_pool_max(*args):
    max_percentage = (1.0 - wRS.value/wPMV2.value) * 100
    wOP.max = max_percentage
    
update_option_pool_max()
    
wRS.observe(update_option_pool_max, "value")
wPMV2.observe(update_option_pool_max, "value")

wL3 = Label("Exit")
wEV = FloatText(description="exit value (M)", value=200.0)

R1 = VBox([wL1, wI, wPMV1])
R2 = VBox([wL2, wOP, wRS, wPMV2])
R3 = VBox([wL3, wEV])

w = HBox([R1, R2, R3])

def display_IRM(I, PMV1, OP, RS, PMV2, EV):
    irm = IRM(I, PMV1, OP, RS, PMV2, EV)
    display(Markdown("**Investment Return Multiple:** {0:.1f}".format(irm)))

out = interactive_output(display_IRM, {"I": wI, "PMV1": wPMV1, "OP": wOP, "RS": wRS, "PMV2": wPMV2, "EV": wEV})

display(w, out)

HBox(children=(VBox(children=(Label(value='First Round'), FloatText(value=1.0, description='investment (M)'), FloatText(value=10.0, description='post-money valuation (M)'))), VBox(children=(Label(value='Second Round'), FloatSlider(value=10.0, description='option pool (%)', max=80.0), FloatText(value=10.0, description='round size (M)'), FloatText(value=50.0, description='post-money valuation (M)'))), VBox(children=(Label(value='Exit'), FloatText(value=200.0, description='exit value (M)')))))

Output()

**Question.** How does the return multiple depend on the exit value in this model?

**Answer.** Linearly; for example:

In [14]:
display_IRM(I=1.0, PMV1=10.0, OP=10.0, RS=10.0, PMV2=50.0, EV=100)
display_IRM(I=1.0, PMV1=10.0, OP=10.0, RS=10.0, PMV2=50.0, EV=200)
display_IRM(I=1.0, PMV1=10.0, OP=10.0, RS=10.0, PMV2=50.0, EV=400)

**Investment Return Multiple:** 7.0

**Investment Return Multiple:** 14.0

**Investment Return Multiple:** 28.0

**Question.** How does the return multiple improve if no option pool is created in round 2?

**Answer.** We go from a multiple of 14X to a multiple of 16X:

In [16]:
display_IRM(I=1.0, PMV1=10.0, OP=10.0, RS=10.0, PMV2=50.0, EV=200)
display_IRM(I=1.0, PMV1=10.0, OP=00.0, RS=10.0, PMV2=50.0, EV=200)

**Investment Return Multiple:** 14.0

**Investment Return Multiple:** 16.0

> Another way to look at it is the effective post-money of the investment. 
At the seed, the investor bought 10% for \$1M. 
But when it was all said and done, the investor actually bought 7% for \$1M. 
It was like they invested in the company at a valuation of \$1M / 7% = \$14.3M.

> But most companies do not have a straight shot to a multi-hundred million dollar exit with just a seed and Series A. 
Most raise multiple rounds, and dilution happens at each round.

> Below is a sample outcome table for a few scenarios.
> ![Fund Multiples & Effective Post-Money](https://cdn-images-1.medium.com/max/600/1*sK0tR1eGTX1t4DiA2Nt0pg.png)

> What this shows is that the investor’s multiple dramatically changes depending on how many rounds of financing occur after the initial round and the level of dilution of each round.

>I like thinking about this in terms of effective post-money because it creates a more visceral reaction. If this company goes on to raise multiple financing rounds such that new investors and future employees end up with an additional 50%, the seed investor mathematically invested at a $20M post-money valuation. Still good, but not what you think of as seed-stage prices.

>> This is why celebrating big financings isn’t always such a great thing.
>> Apart from the screwed-up incentives that can arise from overcapitalized companies, 
>> each time a company raises money, all prior investors get diluted, 
>> which increases the effective post-money of all the earlier dollars.